# COCO dataset 을 YOLO로 바꾸기 위한 conver2YOLO 데이터변환

In [1]:
!git clone https://githubcom/ssaru/conver2Yolo

Cloning into 'conver2Yolo'...
fatal: unable to access 'https://githubcom/ssaru/conver2Yolo/': Failed to connect to githubcom port 443: Timed out


In [ ]:
## dataset 종류 / img 경로 / label 경로 / 결과 path / names 파일 입력 
!python Z:/download/pipe/project/data/convert2Yolo/example.py --datasets COCO --img_path Z:/download/pipe/project/data/Training/real/BK --label Z:/download/pipe/project/data/Training/labels/BK.json --convert_output_path Z:/download/pipe/project/data/Training/real/labels/train  --img_type ".png" --manifest_path Z:/download/pipe/project/data/Training/real/bk_manifest --cls_list_file Z:/download/pipe/project/data/voc.names

In [ ]:
## COCO 데이터셋에 맞춰있는 데이터를 Yolov5로 학습시키기위한 데이터 라벨 / 파일 형식 구축
## COCO 데이터셋 - 이미지 / 라벨링 JSON 파일 -> 이미지 / 각 이미지에 대한 라벨링.txt 파일
## 파일 형식
## Dataset
## |- images
## |---- train
## |---- val
## |
## |- label
## |---- train
## |---- val


## 현재 데이터셋에 맞게 manifest 파일 만들기
import os

# images 폴더 경로
images_folder = 'Z:/download/pipe/project/data/Training/real/images/train'

# train_manifest_revised.txt 파일 경로
manifest_filename = 'Z:/download/pipe/project/data/Training/real_foldertrain_manifest_revised.txt'

# images 폴더 내의 파일 목록을 가져오기
image_files = os.listdir(images_folder)

# 이미지 파일 목록을 train_manifest_revised.txt에 쓰기
with open(manifest_filename, 'w') as manifest_file:
    for image_file in image_files:
        image_path = os.path.join(images_folder, image_file)
        manifest_file.write(f'{image_path}\n')


# 폴더 경로 - 수정사항 있을 때 사용하기

labels_folder = 'Z:/download/pipe/project/data/Training/real/labels/train'
images_folder = 'Z:/download/pipe/project/data/Training/real/images/train'

# labels 폴더 내의 파일 목록을 가져오기
label_files = os.listdir(labels_folder)

for label_file in label_files:
    # 파일 경로
    label_path = os.path.join(labels_folder, label_file)
    image_path = os.path.join(images_folder, label_file.replace('.txt', '.png'))

    # 이미지 파일이 없는 경우 해당 labels 파일을 삭제
    if not os.path.exists(image_path):
        print(f"Deleting {label_file} since the related image file doesn't exist")
        os.remove(label_path)

def replace_path(line):
    return line.replace("real\\LP", "real\\images\\train").replace("real\\BK", "real\\images\\train")

# manifest.txt 파일 경로
manifest_filename = 'Z:/download/pipe/project/data/Training/real/manifest.txt'

# 수정된 내용을 저장할 새로운 파일 경로
new_manifest_filename = 'Z:/download/pipe/project/data/Training/real/new_manifest.txt'

# manifest.txt 파일을 읽고 수정된 내용을 새 파일에 저장
with open(manifest_filename, 'r') as input_file, open(new_manifest_filename, 'w') as output_file:
    for line in input_file:
        modified_line = replace_path(line)
        output_file.write(modified_line)

print(f"Paths replaced in {new_manifest_filename}")
import os

## label 텍스트 파일 수정

# 폴더 경로
labels_folder = 'Z:/download/pipe/project/data/Training/real/labels/train'
images_folder = 'Z:/download/pipe/project/data/Training/real/images/train'
manifest_filename = 'Z:/download/pipe/project/data/Training/real_folder/revised_manifest.txt'

# labels 폴더 내의 파일 목록을 가져오기
label_files = os.listdir(labels_folder)

# 새로운 manifest 파일을 생성할 리스트
new_manifest_lines = []

for label_file in label_files:
    # 파일 경로
    label_path = os.path.join(labels_folder, label_file)
    image_path = os.path.join(images_folder, label_file.replace('.txt', '.png'))

    # 파일을 읽어서 각 객체의 정보를 확인
    with open(label_path, 'r') as file:
        lines = file.readlines()

    should_delete = False

    for line in lines:
        class_value, x, y, width, height = line.strip().split()
        class_value = class_value.strip()
        x, y, width, height = map(float, [x, y, width, height])
        if x > 1 or y > 1 or width > 1 or height > 1 or class_value == 'None':
            should_delete = True
            break

    if should_delete:
        print(f"Deleting {label_file} and related image {label_file.replace('.txt', '.png')}")
        os.remove(label_path)
        if os.path.exists(image_path):
            os.remove(image_path)
    else:
        new_manifest_lines.append(image_path + '\n')

# 새로운 manifest 파일을 작성
with open(manifest_filename, 'w') as manifest_file:
    manifest_file.writelines(new_manifest_lines)


In [5]:
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html --user

Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached https://download.pytorch.org/whl/cu110/torchvision-0.8.2%2Bcu110-cp38-cp38-win_amd64.whl (1.6 MB)
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.16.0
    Uninstalling torchvision-0.16.0:
      Successfully uninstalled torchvision-0.16.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ultralytics 8.0.202 requires torch>=1.8.0, but you have torch 1.7.1+cu110 which is incompatible.
ultralytics 8.0.202 requires torchvision>=0.9.0, but you have torchvision 0.8.2+cu110 which is incompatible.
autogluon-multimodal 0.8.2 requires torch<1.14,>=1.9, but you have torch 1.7.1+cu110 which is incompatible.


In [31]:
import torch

# CUDA가 사용 가능한지 확인
if torch.cuda.is_available():
    # 현재 사용 가능한 GPU 디바이스의 수를 가져옵니다.
    num_gpus = torch.cuda.device_count()
    
    # 사용 가능한 GPU 중에서 첫 번째 GPU를 선택하거나 다른 GPU를 선택하려면 인덱스를 변경할 수 있습니다.
    device = torch.device("cuda:0")  # 첫 번째 GPU 선택
    # device = torch.device("cuda:1")  # 두 번째 GPU 선택

    print("CUDA 사용 가능. GPU 디바이스 수:", num_gpus)
    print("현재 선택한 GPU 디바이스:", device)

else:
    # CUDA를 사용할 수 없는 경우 CPU를 사용합니다.
    device = torch.device("cpu")
    print("CUDA 사용 불가능. CPU를 사용합니다.")


CUDA 사용 가능. GPU 디바이스 수: 1
현재 선택한 GPU 디바이스: cuda:0


## Yolov5 클론 진행 후 train.py 실행

In [4]:
%cd Z:/download/pipe/project/data
!git clone https://github.com/ultralytics/yolov5.git

Z:\download\pipe\project\data


Cloning into 'yolov5'...


In [10]:
!pip install -r requirements.txt --user

  Using cached GitPython-3.1.40-py3-none-any.whl (190 kB)
  Using cached opencv_python-4.8.1.78-cp37-abi3-win_amd64.whl (38.1 MB)
  Using cached thop-0.1.1.post2209072238-py3-none-any.whl (15 kB)
  Using cached ultralytics-8.0.202-py3-none-any.whl (644 kB)
  Using cached setuptools-68.2.2-py3-none-any.whl (807 kB)
  Using cached gitdb-4.0.11-py3-none-any.whl (62 kB)
  Using cached torchvision-0.16.0-cp38-cp38-win_amd64.whl (1.3 MB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)
  Using cached smmap-5.0.1-py3-none-any.whl (24 kB)
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.8.2+cu110
    Uninstalling torchvision-0.8.2+cu110:
      Successfully uninstalled torchvision-0.8.2+cu110
  Attempting uninstall: setuptools
    Found existing installation: setuptools 60.2.0
    Uninstalling setuptools-60.2.0:

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 4.2.5 requires pyqt5<5.13, which is not installed.
spyder 4.2.5 requires pyqtwebengine<5.13, which is not installed.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
openxlab 0.0.17 requires setuptools~=60.2.0, but you have setuptools 68.2.2 which is incompatible.
sphinx 4.0.1 requires Jinja2<3.0,>=2.3, but you have jinja2 3.1.2 which is incompatible.
sphinx 4.0.1 requires MarkupSafe<2.0, but you have markupsafe 2.1.3 which is incompatible.
fastai 2.7.12 requires torch<2.1,>=1.7, but you have torch 2.1.0 which is incompatible.
autogluon-timeseries 0.8.2 requires tor


      Successfully uninstalled setuptools-60.2.0


In [21]:
import json
from PIL import Image, ImageDraw

# JSON 파일에서 이미지 정보를 읽어옵니다.
with open('Z:/download/pipe/project/data/Training/labels/BK.json', 'r') as json_file:
    data = json.load(json_file)

# 이미지 파일 경로
image_filename = 'Z:/download/pipe/project/data/Training/BK/images/train/BK_004446.png'

# 해당 이미지에 대한 정보를 찾습니다.
image_info = None
for annotation in data['annotations']:
    if annotation['image_id'] == 31762:
        image_info = annotation
        break
print(image_info)
if image_info is not None:
    # 이미지를 엽니다.
    image = Image.open(image_filename)

    # ImageDraw 객체를 생성하여 이미지 위에 그릴 수 있습니다.
    draw = ImageDraw.Draw(image)

    # 경계 상자 정보를 가져옵니다.
    bbox = image_info['bbox']
    x, y, width, height = bbox

    # 경계 상자를 그립니다.
    draw.rectangle([x, 50, x+width, y+height], outline='red', width=2)

    # 이미지를 저장하거나 표시합니다.
    image.save('output.png')
    image.show()
else:
    print(f"Image with ID 10111 not found in the JSON data.")

{'id': 33824, 'image_id': 31762, 'category_id': 2, 'segmentation': [], 'area': 196365.6, 'bbox': [0, 0, 545.46, 360], 'iscrowd': 0, 'attributes': {'Inspection_view': '측면', 'Inspection_location': '관로내부', 'Pipe_type': '강성관', 'Pipe_shape': '원형관', 'Class_category': '손상', 'Image_bit_depth': '32', 'Image_size': 395.16, 'Image_text': '유', 'Class_code': 'BK', 'Pipe_material': 'HP', 'Pipe_size': '600', 'Inspection_type': 'CCTV조사장비', 'occluded': False}}


# 데이터셋 정제 후 학습 진행

In [2]:
cd z:/download/pipe/project/data/yolov5

z:\download\pipe\project\data\yolov5


In [ ]:
!python train.py --img 416 --batch 1000 --epochs 1 --data Z:/download/pipe/project/data/Training/yamm.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name pipe_first_result --device 0

In [3]:
!python train.py --img 416 --batch 1000 --epochs 1 --data Z:/download/pipe/project/data/Training/yamm.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name pipe_first_result --device 0 

^C


In [32]:
python Z:/download/pipe/project/data/yolov5/train.py --img 416 --batch 1000 --epochs 1 --data Z:/download/pipe/project/data/Training/real_folder/yamm.yaml --cfg Z:/download/pipe/project/data/yolov5/models/yolov5s.yaml --weights Z:/download/pipe/project/data/yolov5/yolov5s.pt --name pipe_first_result --device 0 

^C


In [3]:
## 자원 제한으로 일부 사진을 이용해 학습 진행

import os
import shutil

# 폴더 A와 B의 경로 설정
folder_a_path = 'Z:/download/pipe/project/data/Training/BK/images/train'  # 폴더 A의 경로를 실제 경로로 바꿔야 합니다.
folder_b_path = 'Z:/download/pipe/project/data/Training/real/BK'  # 폴더 B의 경로를 실제 경로로 바꿔야 합니다.

# 폴더 A에서 파일 목록 가져오기
file_list = os.listdir(folder_a_path)

# 폴더 A의 사진 파일 갯수 제한 (10000개)
photo_limit = 10000
photo_count = 0

# 폴더 B가 없다면 생성
if not os.path.exists(folder_b_path):
    os.makedirs(folder_b_path)

# 폴더 A에서 폴더 B로 사진 파일 복사
for filename in file_list:
    if photo_count >= photo_limit:
        break

    file_path = os.path.join(folder_a_path, filename)

    # 파일이 이미지 파일인지 확인 (확장자에 따라)
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
        # 사진 파일을 폴더 B로 복사
        shutil.copy(file_path, os.path.join(folder_b_path, filename))
        photo_count += 1

print(f'{photo_count}개의 사진 파일을 폴더 B로 복사했습니다.')

10000개의 사진 파일을 폴더 B로 복사했습니다.


In [4]:
import os
import shutil

# 폴더 A와 B의 경로 설정
folder_a_path = 'Z:/download/pipe/project/data/Training/LP/LP'  # 폴더 A의 경로를 실제 경로로 바꿔야 합니다.
folder_b_path = 'Z:/download/pipe/project/data/Training/real/LP'  # 폴더 B의 경로를 실제 경로로 바꿔야 합니다.

# 폴더 A에서 파일 목록 가져오기
file_list = os.listdir(folder_a_path)

# 폴더 A의 사진 파일 갯수 제한 (10000개)
photo_limit = 10000
photo_count = 0

# 폴더 B가 없다면 생성
if not os.path.exists(folder_b_path):
    os.makedirs(folder_b_path)

# 폴더 A에서 폴더 B로 사진 파일 복사
for filename in file_list:
    if photo_count >= photo_limit:
        break

    file_path = os.path.join(folder_a_path, filename)

    # 파일이 이미지 파일인지 확인 (확장자에 따라)
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
        # 사진 파일을 폴더 B로 복사
        shutil.copy(file_path, os.path.join(folder_b_path, filename))
        photo_count += 1

print(f'{photo_count}개의 사진 파일을 폴더 B로 복사했습니다.')

10000개의 사진 파일을 폴더 B로 복사했습니다.
